In [ ]:
from transformers import MarianMTModel, MarianTokenizer
from transformers import pipeline
import torch
import torch.nn.functional as F
from torch.optim import AdamW
from copy import deepcopy
import numpy as np
from tqdm import tqdm
import os
import random
import gc
import json

In [ ]:
def get_model():
    model_name = "Helsinki-NLP/opus-mt-en-hi"
    tokenizer = MarianTokenizer.from_pretrained(model_name)
    model = MarianMTModel.from_pretrained(model_name)

    device = torch.device("mps" if torch.mps.is_available() else "cpu")
    model.to(device)

    return model, tokenizer

In [ ]:
# device = torch.device("mps" if torch.mps.is_available() else "cpu")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.to(device)

In [ ]:
with open('output_with_train_split.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

en_train = []
hi_train = []
en_test = []
hi_test = []
hi_test_names = []

for document in data:
    en_doc = [sent['english'] for sent in document['sentences']]
    hi_doc = [sent['hindi'] for sent in document['sentences']]

    if document['is_train']:
        en_train.append(en_doc)
        hi_train.append(hi_doc)
    else:
        en_test.append(en_doc)
        hi_test.append(hi_doc)
        hi_test_names.append(document['doc_name'].replace('.txt',''))

print(len(en_train))
print(len(hi_train))
print(len(en_test))
print(len(hi_test))

1277
1277
450
450


In [ ]:
model, tokenizer = get_model()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/812k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/1.07M [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.10M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/306M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/306M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [ ]:
def write_translations(model, tokenizer, sentences, docName=""):
    model.eval()
    with torch.no_grad():
        inputs = tokenizer(sentences, return_tensors="pt", padding=True).to(model.device)
        outputs = model.generate(**inputs)
        translations = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        with open(f"baseline-translations_{docName}.txt", "w") as f:
            for hi in translations:

                f.write(f"{hi}\n")

In [ ]:
for idx, doc in tqdm(enumerate(en_test), desc='Writing translations...', total=len(en_test), ncols=80):
    write_translations(model, tokenizer, doc, hi_test_names[idx])

Writing translations...: 450it [1:00:03,  8.01s/it]


In [ ]:
!zip -r translations.zip baseline-translations_*.txt

  adding: baseline-translations_agreement-between-india-and-maldives-for-avoidance-of-double-taxation-of-income-from-international-air-transport.txt (deflated 67%)
  adding: baseline-translations_approval-for-phase-ii-of-ecourts-mission-mode-project.txt (deflated 71%)
  adding: baseline-translations_artwork-made-with-pearls-presented-to-pm.txt (deflated 60%)
  adding: baseline-translations_bangladesh-home-minister-calls-on-prime-minister.txt (deflated 66%)
  adding: baseline-translations_bhutanese-parliamentary-delegation-calls-on-pm.txt (deflated 69%)
  adding: baseline-translations_cabinet-apprised-of-framework-agreement-between-india-and-vietnam-on-cooperation-in-the-exploration-and-uses-of-outer-space-for-peaceful-purposes.txt (deflated 72%)
  adding: baseline-translations_cabinet-apprised-of-mou-between-india-and-france-on-technical-cooperation-in-the-field-of-railways.txt (deflated 74%)
  adding: baseline-translations_cabinet-apprised-of-mou-between-india-and-morocco-on-cooperati

In [ ]:
from google.colab import files
files.download("translations.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>